<a href="https://colab.research.google.com/github/yneha70/IIITH/blob/main/Task_3_Object_Detection_and_Sementation_Video_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import subprocess
from ultralytics import YOLO

BASE_DIR = r"C:\Users\Neha\Desktop\YOLO_Restaurant_Project"
input_video = os.path.join(BASE_DIR, "input", "input_video.mp4")
frames_folder = os.path.join(BASE_DIR, "frames")
output_frames_folder = os.path.join(BASE_DIR, "output_frames")
final_output_video = os.path.join(BASE_DIR, "output_video.mp4")

os.makedirs(frames_folder, exist_ok=True)
os.makedirs(output_frames_folder, exist_ok=True)

print(" Extracting 24 frames per second from video...")
extract_cmd = [
    r"C:\ffmpeg\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe",
    "-i", input_video,
    "-vf", "fps=24",
    os.path.join(frames_folder, "frame_%04d.jpg"),
    "-hide_banner", "-loglevel", "error"
]
subprocess.run(extract_cmd)
print(" Frames extracted successfully and saved to:", f
print("\n Running YOLO object detection on frames...")
model = YOLO("yolov8l-seg.pt")
CLASS_COLORS = {
    "person": (0, 255, 0),
    "car": (255, 0, 0),
    "truck": (255, 128, 0),
    "bus": (0, 255, 255),
    "motorbike": (255, 0, 255),
    "bicycle": (0, 128, 255),
    "traffic light": (0, 0, 255),
    "stop sign": (128, 0, 128),
    "dog": (160, 82, 45),
    "cat": (255, 105, 180),
}

DEFAULT_COLOR = (200, 200, 200)

frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith((".jpg", ".png"))])
for idx, frame_name in enumerate(frame_files, start=1):
    frame_path = os.path.join(frames_folder, frame_name)
    img = cv2.imread(frame_path)

    if img is None:
        print(f" Skipping unreadable frame: {frame_name}")
        continue

    results = model.predict(source=img, conf=0.25, save=False)

    for box, cls in zip(results[0].boxes, results[0].boxes.cls):
        class_id = int(cls)
        class_name = model.names[class_id].lower()
        conf = float(box.conf)
        xyxy = [int(x) for x in box.xyxy.tolist()[0]]

        color = CLASS_COLORS.get(class_name, DEFAULT_COLOR)
        label = f"{class_name.capitalize()} {conf:.2f}"

        cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)

        (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(img, (xyxy[0], xyxy[1] - text_h - 6),
                      (xyxy[0] + text_w, xyxy[1]), color, -1)
        cv2.putText(img, label, (xyxy[0], xyxy[1] - 4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    output_path = os.path.join(output_frames_folder, f"processed_{frame_name}")
    cv2.imwrite(output_path, img)
    print(f" Processed frame {idx}/{len(frame_files)}: {frame_name}")

print("\n All frames processed and saved to:", output_frames_folder)

print("\n Stitching processed frames into final output video...")
combine_cmd = [
    r"C:\ffmpeg\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe",
    "-framerate", "24",
    "-i", os.path.join(output_frames_folder, "processed_frame_%04d.jpg"),
    "-c:v", "libx264",
    "-pix_fmt", "yuv420p",
    final_output_video,
    "-hide_banner", "-loglevel", "error"
]
subprocess.run(combine_cmd)
print(" Output video created successfully at:", final_output_video)
print("\n Task complete! You can now compare input and output videos.")
